- splitting up users (50% of user 1 rating, 50% of user 2 rating)


1) reassign timestamps arbitrarily 
2) for each user, take first 1/2 --> als, 1/2 --> stream/test

1) pretraining movie vectors --> 

does using pyspark still count --> yes, keep ML simple for system paper
maybe point query slows down enough? --> maybe a bad thing

1) verify model that works 
2) verify updating user embeddings helps predictions
3) can figure out slowing down later
4) are we prioritizng larger workloads (1M instead of 100k) --> 

push to main instead of branch (movie-lens/als)
- fix up joey als

undergrad onboarding
- open source side


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master('local').appName('als').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/10 01:44:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [64]:
dir_path = "/Users/amitnarang/movielens/5050/"

In [ ]:
print(dir_path)

In [32]:
from pyspark.sql.types import StructField,IntegerType, StructType,StringType
newDF=[StructField('user_id',IntegerType(),True),
       StructField('movie_id',IntegerType(),True),
       StructField('rating',IntegerType(),True),
       StructField('timestamp',IntegerType(),True),
       ]  
finalStruct=StructType(fields=newDF)
ratings = spark.read.csv(dir_path + "als_data.csv", schema=finalStruct, header=True)
test_data = spark.read.csv(dir_path + "stream_data.csv", schema=finalStruct, header=True)

In [33]:
ratings.show()

+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|    259|     255|     4|874724710|
|    259|     286|     4|874724727|
|    259|     298|     4|874724754|
|    259|     185|     4|874724781|
|    259|     173|     4|874724843|
|    259|     772|     4|874724882|
|    259|     108|     4|874724882|
|    259|     288|     3|874724905|
|    259|     928|     4|874724937|
|    259|     117|     4|874724988|
|    259|     200|     4|874725081|
|    259|     405|     3|874725120|
|    259|    1074|     3|874725264|
|    259|     176|     4|874725386|
|    259|     210|     4|874725485|
|    259|     357|     5|874725485|
|    851|     687|     2|874728168|
|    851|     696|     3|874728338|
|    851|     284|     3|874728338|
|    851|     295|     5|874728370|
+-------+--------+------+---------+
only showing top 20 rows



In [34]:
test_data.sort('user_id').show()

+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|      1|     176|     5|876892468|
|      1|     231|     1|876893031|
|      1|      91|     5|876892636|
|      1|     193|     4|876892654|
|      1|     217|     3|876892676|
|      1|     177|     5|876892701|
|      1|     216|     5|876892701|
|      1|     194|     4|876892743|
|      1|      73|     3|876892774|
|      1|      59|     5|876892817|
|      1|      41|     2|876892818|
|      1|     133|     4|876892818|
|      1|     195|     5|876892855|
|      1|     170|     5|876892856|
|      1|     218|     3|876892856|
|      1|     213|     2|876892896|
|      1|     157|     4|876892918|
|      1|     223|     5|876892918|
|      1|      27|     2|876892946|
|      1|     227|     4|876892946|
+-------+--------+------+---------+
only showing top 20 rows



In [35]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [36]:
als = ALS(
         userCol="user_id", 
         itemCol="movie_id",
         ratingCol="rating", 
         nonnegative = True, 
         implicitPrefs = False,
         coldStartStrategy="drop"
)

In [37]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

In [38]:
# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [150]) \
            .addGrid(als.regParam, [.1]) \
            .build()

In [39]:
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(
           metricName="rmse", 
           labelCol="rating", 
           predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  1


In [40]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

In [41]:
#Fit cross validator to the 'train' dataset
model = cv.fit(ratings)
#Extract best model from the cv model above
best_model = model.bestModel
# View the predictions
test_predictions = best_model.transform(test_data)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

1.0037441470066895


In [42]:
print("**Best Model**")
# Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())
# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())
# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())



**Best Model**
  Rank: 150
  MaxIter: 10
  RegParam: 0.1


In [43]:
recommendations = best_model.recommendForAllUsers(5)
recommendations.show()

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|      1|[{408, 4.934828},...|
|      2|[{1449, 4.4305215...|
|      3|[{344, 3.3770661}...|
|      4|[{1131, 5.6823254...|
|      5|[{1142, 4.613391}...|
|      6|[{134, 4.254445},...|
|      7|[{612, 4.9064746}...|
|      8|[{127, 4.917586},...|
|      9|[{1131, 4.366975}...|
|     10|[{1449, 4.9595895...|
|     11|[{718, 4.5692844}...|
|     12|[{318, 4.9410357}...|
|     13|[{851, 4.9060307}...|
|     14|[{408, 5.027333},...|
|     15|[{459, 4.47861}, ...|
|     16|[{136, 4.944799},...|
|     17|[{483, 4.0543947}...|
|     18|[{170, 4.6695433}...|
|     19|[{8, 4.571447}, {...|
|     20|[{916, 4.141518},...|
+-------+--------------------+
only showing top 20 rows



In [44]:
from pyspark.sql.functions import explode, col
nrecommendations = recommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('user_id', col("rec_exp.movie_id"), col("rec_exp.rating"))
nrecommendations.limit(10).show()

+-------+--------+---------+
|user_id|movie_id|   rating|
+-------+--------+---------+
|      1|     408| 4.934828|
|      1|    1142| 4.832595|
|      1|     169|4.7931576|
|      1|     114|4.7699485|
|      1|      50| 4.749361|
|      2|    1449|4.4305215|
|      2|     275|4.2517853|
|      2|     190|4.2478666|
|      2|     483| 4.224928|
|      2|     478| 4.186218|
+-------+--------+---------+



In [45]:
nrecommendations.filter('user_id = 100').show()

+-------+--------+---------+
|user_id|movie_id|   rating|
+-------+--------+---------+
|    100|     313|4.2239876|
|    100|     916|3.9103782|
|    100|     318|3.8815289|
|    100|      22|3.8615365|
|    100|     900|3.8392437|
+-------+--------+---------+



In [47]:
best_model.itemFactors.toPandas().sort_values('id').to_csv(dir_path + 'movie_features.csv', index = False)

In [48]:
best_model.userFactors.toPandas().sort_values('id').to_csv(dir_path + 'user_features.csv', index = False)

In [56]:
predictions = best_model.transform(test_data)
evaluator = RegressionEvaluator().setMetricName("rmse").setLabelCol("rating").setPredictionCol("prediction")
rmse = evaluator.evaluate(predictions)

In [58]:
print(rmse)

1.0037441470066895
